In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [14]:
globals()['models_loaded'] = False
globals()['my_model'] = None

# Toy predict function. Normally you'd use your loaded globals()['my_model'] here
def predict(df):
    return 'predicted-name-of-channel'

predict_udf = udf(predict, StringType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    print(type(df))
    # Utilize our predict function
    df_withpreds = df.withColumn("pred", predict_udf(
        struct([df[x] for x in df.columns])
    ))
    df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = '***' # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    # df_result = globals()['my_model'].transform(df)
    # df_result.show()

In [12]:
ssc = StreamingContext(sc, 10)

In [15]:
lines = ssc.socketTextStream("localhost", 8080)
lines.foreachRDD(process)

In [16]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2022-05-21 00:16:00 =========
+-------+--------------------+--------------------+--------+
|channel|            datetime|             message|username|
+-------+--------------------+--------------------+--------+
|   #pgl|2022-05-20T22:15:...|jabbi looks like ...|   sgaag|
+-------+--------------------+--------------------+--------+

+-------+--------------------+--------------------+--------+--------------------+
|channel|            datetime|             message|username|                pred|
+-------+--------------------+--------------------+--------+--------------------+
|   #pgl|2022-05-20T22:15:...|jabbi looks like ...|   sgaag|predicted-name-of...|
+-------+--------------------+--------------------+--------+--------------------+

========= 2022-05-21 00:16:00 =========
+-------+--------------------+--------------------+--------+
|channel|            datetime|             message|username|
+-------+--------------------+--------------------+--------+
|   #pgl|2022-05-20T

In [17]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
========= 2022-05-21 00:16:10 =========
+----------+--------------------+--------------------+-------------+
|   channel|            datetime|             message|     username|
+----------+--------------------+--------------------+-------------+
|#iitztimmy|2022-05-20T22:15:...|BatChest offline ...|       fengoe|
|      #pgl|2022-05-20T22:15:...|My future one nig...|skinnysicario|
+----------+--------------------+--------------------+-------------+

+----------+--------------------+--------------------+-------------+--------------------+
|   channel|            datetime|             message|     username|                pred|
+----------+--------------------+--------------------+-------------+--------------------+
|#iitztimmy|2022-05-20T22:15:...|BatChest offline ...|       fengoe|predicted-name-of...|
|      #pgl|2022-05-20T22:15:...|My future one nig...|skinnysicario|predicted-name-of...|
+----------+--------------------+----------

In [18]:
print('completed')

completed
